**概要**  
- LightGBM
- カテゴリカル変数の処理なし
- 欠損値処理
    - 量的変数・質的変数それぞれ一律で同じ値を代入
- 特徴量
    - 河原くんが調べた特徴量をいくつか追加
- ハイパーパラメータチューニング
    - Optunaを用いたベイズ最適化
    - 探索範囲は既定のものを使用

In [1]:
# # ライブラリの読み込み
# !pip install /kaggle/input/cibmtr-whl-files-for-installation/autograd-1.7.0-py3-none-any.whl
# !pip install /kaggle/input/cibmtr-whl-files-for-installation/autograd-gamma-0.5.0.tar.gz
# !pip install /kaggle/input/cibmtr-whl-files-for-installation/interface_meta-1.3.0-py3-none-any.whl
# !pip install /kaggle/input/cibmtr-whl-files-for-installation/formulaic-1.1.1-py3-none-any.whl
# !pip install /kaggle/input/cibmtr-whl-files-for-installation/lifelines-0.30.0-py3-none-any.whl

# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings
# warnings.filterwarnings('ignore')
# import lifelines
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# from catboost import CatBoostClassifier, CatBoostRegressor
# from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from lifelines import KaplanMeierFitter
# from sklearn.metrics import mean_squared_error
# import warnings
# # ライブラリのインストール
# from sklearn.model_selection import StratifiedKFold
# import lightgbm as lgb

In [2]:
# # データの読み込み

# df = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
# df_test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
# data_dic = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv")# 辞書データ
# sample_sub = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv")

In [3]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [4]:
import lifelines
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from lifelines import KaplanMeierFitter
from sklearn.metrics import mean_squared_error
import warnings

In [ ]:
# データの読み込み
# INPUT_DIRにディレクトリを指定
INPUT_DIR = "xxx"

df_train = pd.read_csv(INPUT_DIR + "train.csv")
df_test = pd.read_csv(INPUT_DIR + "test.csv")
data_dic = pd.read_csv(INPUT_DIR + "data_dictionary.csv")# 辞書データ
sample_sub = pd.read_csv(INPUT_DIR + "sample_submission.csv")
df_train.head(20)

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223
5,5,High,No,Poor,Yes,2.0,7.0,TBI + Cy +- Other,No,4.0,...,Permissive mismatched,Related,"N/A, Mel not given",5.0,No,2.0,Yes,6.0,1.0,7.095
6,6,Low,No,Poor,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,0.0,46.464
7,7,High,No,NaN,Not done,2.0,5.0,TBI + Cy +- Other,No,3.0,...,NaN,Related,"N/A, Mel not given",5.0,No,1.0,No,6.0,0.0,18.076
8,8,Intermediate,No,Other,No,NaN,NaN,TBI + Cy +- Other,No,6.0,...,NaN,Related,"N/A, Mel not given",8.0,No,NaN,Yes,10.0,1.0,10.130
9,9,Intermediate,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,GvH non-permissive,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,9.0,1.0,5.434


**生存関数に関して**  
- Kaplan-Meier推定量を用いた生存関数の推定  
    - 「生存期間」と「生存フラグ」から『特定の期間における生存確率』を算出
    - 今回はこの『生存確率』が目的変数
- 参考：https://qiita.com/roki18d/items/3d725333ff1616a8977c

In [6]:
# 生存確率を計算する関数
# efs：無イベント生存率
# efs_time：無イベント生存までの時間
def transform_survival_probability(df, time_col='efs_time', event_col='efs'):
    kmf = KaplanMeierFitter() # インスタンスを作成
    kmf.fit(df[time_col], event_observed=df[event_col]) # カプラン・マイヤー推定量にデータをフィット
    survival_probabilities = kmf.survival_function_at_times(df[time_col]).values.flatten()
    censored_mask = df[event_col] == 0
    return survival_probabilities

In [7]:
# 生存確率を目的変数として代入
df_train["target"] = transform_survival_probability(df_train, time_col='efs_time', event_col='efs')

In [8]:
# 不要になったカラムを削除
drop_cols = ["ID", 'efs', 'efs_time']
df_train = df_train.drop(columns=[col for col in drop_cols if col in df_train.columns])
df_test = df_test.drop(columns=[col for col in drop_cols if col in df_test.columns])  # "df_test"には'efs'と'efs_time'はない

In [9]:
df_train.head()

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,target
0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,Bone marrow,...,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.458687
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,0.847759
2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.462424
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.456661
4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,No,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.464674


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Data columns (total 58 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   dri_score               28646 non-null  object 
 1   psych_disturb           26738 non-null  object 
 2   cyto_score              20732 non-null  object 
 3   diabetes                26681 non-null  object 
 4   hla_match_c_high        24180 non-null  float64
 5   hla_high_res_8          22971 non-null  float64
 6   tbi_status              28800 non-null  object 
 7   arrhythmia              26598 non-null  object 
 8   hla_low_res_6           25530 non-null  float64
 9   graft_type              28800 non-null  object 
 10  vent_hist               28541 non-null  object 
 11  renal_issue             26885 non-null  object 
 12  pulm_severe             26665 non-null  object 
 13  prim_disease_hct        28800 non-null  object 
 14  hla_high_res_6          23516 non-null

**欠損値処理**

In [11]:
# 欠損値には一律で同じ値を代入
def replace_nulls_with_default(df, float_default=0.0, object_default="Unknown"):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].fillna(float_default)
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].fillna(object_default)
    return df

In [12]:
df_train = replace_nulls_with_default(df_train, float_default=-1.0, object_default="Missing")
df_test = replace_nulls_with_default(df_test, float_default=-1.0, object_default="Missing")
df_train.isnull().sum()

dri_score                 0
psych_disturb             0
cyto_score                0
diabetes                  0
hla_match_c_high          0
hla_high_res_8            0
tbi_status                0
arrhythmia                0
hla_low_res_6             0
graft_type                0
vent_hist                 0
renal_issue               0
pulm_severe               0
prim_disease_hct          0
hla_high_res_6            0
cmv_status                0
hla_high_res_10           0
hla_match_dqb1_high       0
tce_imm_match             0
hla_nmdp_6                0
hla_match_c_low           0
rituximab                 0
hla_match_drb1_low        0
hla_match_dqb1_low        0
prod_type                 0
cyto_score_detail         0
conditioning_intensity    0
ethnicity                 0
year_hct                  0
obesity                   0
mrd_hct                   0
in_vivo_tcd               0
tce_match                 0
hla_match_a_high          0
hepatic_severe            0
donor_age           

**データ型処理**

In [13]:
# object型をcategory型に変換
def convert_object_to_category(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype('category')
    return df

df_train = convert_object_to_category(df_train)
df_test = convert_object_to_category(df_test)

In [14]:
df_train.head()

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,target
0,N/A - non-malignant indication,No,Missing,No,-1.0,-1.0,No TBI,No,6.0,Bone marrow,...,No,Missing,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.458687
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,0.847759
2,N/A - non-malignant indication,No,Missing,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.462424
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.456661
4,High,No,Missing,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,No,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.464674


**特徴量の追加**

In [15]:
def eda(df):
    df['age_karnofsky'] = df['age_at_hct'] * df['karnofsky_score']
    df['age_comorbidity'] = df['age_at_hct'] * df['comorbidity_score']
    df['donor_recipient_age_diff'] = abs(df['donor_age'] - df['age_at_hct'])
    df['years_since_2000'] = df['year_hct'] - 2000
    df['hla_match_ratio'] = (df['hla_high_res_8'] + df['hla_low_res_8']) / 16
    df['age_squared'] = df['age_at_hct'] ** 2
    df['karnofsky_squared'] = df['karnofsky_score'] ** 2

    
    # --- 1. 周期的な特徴量 ---
    # sin_year: サイン波
    df['sin_year'] = np.sin(df['year_hct'] * (2 * np.pi) / 100)
    
    # cos_year: コサイン波
    df['cos_year'] = np.cos(df['year_hct'] * (2 * np.pi) / 100)
    
    # --- 2. 年齢に基づく特徴量 ---
    # age_gap: 患者とドナーの年齢差（絶対値）
    df['age_gap'] = np.abs(df['age_at_hct'] - df['donor_age'])
    
    # diff_age_vs_donor: 患者の年齢 - ドナーの年齢
    df['diff_age_vs_donor'] = df['age_at_hct'] - df['donor_age']
    
    # --- 3. 技術進歩に基づく特徴量 ---
    # tech_progress: HCT実施年 - 最古のHCT実施年
    min_year_hct = df['year_hct'].min()
    df['tech_progress'] = df['year_hct'] - min_year_hct
    
    # --- 4. 性別に関する特徴量 ---
    # sex_one: 性別一致情報から患者の性別を抽出
    df['sex_one'] = df['sex_match'].str[0]  # 最初の文字を抽出
    
    # sex_two: 性別一致情報からドナーの性別を抽出
    df['sex_two'] = df['sex_match'].str[2]  # 3番目の文字を抽出
    
    # SameSex: 患者とドナーの性別が一致しているか
    df['SameSex'] = (df['sex_one'] == df['sex_two']).astype(int)

In [16]:
eda(df_train)
eda(df_test)
df_train.head()

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,age_squared,karnofsky_squared,sin_year,cos_year,age_gap,diff_age_vs_donor,tech_progress,sex_one,sex_two,SameSex
0,N/A - non-malignant indication,No,Missing,No,-1.0,-1.0,No TBI,No,6.0,Bone marrow,...,98.843364,8100.0,0.844328,0.535827,10.942,10.942,8,M,F,0
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,1910.127025,8100.0,0.481754,0.876307,28.585,-28.585,0,F,F,1
2,N/A - non-malignant indication,No,Missing,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,1155.796009,8100.0,0.929776,0.368125,34.997,34.997,11,F,M,0
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,1870.130025,8100.0,0.535827,0.844328,14.015,14.015,1,M,M,1
4,High,No,Missing,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,884.467600,8100.0,0.904827,0.425779,27.070,-27.070,10,M,F,0


In [17]:
# # カテゴリごとの条件を指定
# categories = {
#     'high': ['high'],
#     'low': ['low'],
#     'intermediate': ['intermediate'],
#     'tbd': ['tbd'],
#     'missing': ['missing'],
#     'n/a': ['n/a']
# }

# # 小文字に変換し、各カテゴリにマッチするかどうかを判定
# def categorize_dri_score(value):
#     value = value.lower()  # 小文字に変換
#     for category, keywords in categories.items():
#         if value in [kw.lower() for kw in keywords]:
#             return category
#     return 'other'  # 該当しない場合は'other'

# # 新しいカラムを追加して分類結果を保存
# df['dri_score_category'] = df['dri_score'].apply(categorize_dri_score)

# # 結果を表示
# print(df)

In [18]:
df_train = convert_object_to_category(df_train)
df_test = convert_object_to_category(df_test)
df_train.head()

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,age_squared,karnofsky_squared,sin_year,cos_year,age_gap,diff_age_vs_donor,tech_progress,sex_one,sex_two,SameSex
0,N/A - non-malignant indication,No,Missing,No,-1.0,-1.0,No TBI,No,6.0,Bone marrow,...,98.843364,8100.0,0.844328,0.535827,10.942,10.942,8,M,F,0
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,1910.127025,8100.0,0.481754,0.876307,28.585,-28.585,0,F,F,1
2,N/A - non-malignant indication,No,Missing,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,1155.796009,8100.0,0.929776,0.368125,34.997,34.997,11,F,M,0
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,1870.130025,8100.0,0.535827,0.844328,14.015,14.015,1,M,M,1
4,High,No,Missing,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,884.467600,8100.0,0.904827,0.425779,27.070,-27.070,10,M,F,0


In [19]:
# データの準備
X = df_train.drop(columns=['target'], axis=1)
y = df_train['target']
race_groups = df_train['race_group']  # 層別に利用

**モデル構築**  
- LightGBM
    - インポート：https://qiita.com/ryo_hagi/items/343d23904cad570b03b5
    - ハイパラチューニング：https://qiita.com/c60evaporator/items/351188110f328ff921b9
        - 今回は実施できていない
    - 種類と使い方：https://lee-ann-al-chan.com/wp/wp-login.php/2022/10/03/lightgbm_trainingapi_sclearnapi/
        - Training API
            - 今回はこっちを使ってみる
        - Scikit-learn API
- C統計量について
    - 理論上AUCと同じ値
    - 参考：https://note.com/yohei_okada/n/nd384bd62c6d5

In [20]:
# Stratified C-indexの算出
def stratified_c_index(y_true, y_pred, groups):
    unique_groups = np.unique(groups)
    c_indices = []

    for group in unique_groups:
        mask = groups == group
        if sum(mask) > 1:  
            y_true_group = y_true[mask]
            y_pred_group = y_pred[mask]
            concordant = 0
            permissible = 0

            for i in range(len(y_true_group)):
                for j in range(i + 1, len(y_true_group)):
                    if y_true_group[i] != y_true_group[j]:
                        permissible += 1
                        if (y_pred_group[i] > y_pred_group[j] and y_true_group[i] > y_true_group[j]) or \
                           (y_pred_group[i] < y_pred_group[j] and y_true_group[i] < y_true_group[j]):
                            concordant += 1

            c_indices.append(concordant / permissible if permissible > 0 else 0)

    c_indices = np.array(c_indices)
    return np.mean(c_indices) - np.std(c_indices)

In [21]:
# カテゴリカル変数の列を取得
cat_features = list(X.select_dtypes(include=['category']).columns)

In [22]:
# ライブラリのインストール
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import optuna
from sklearn.model_selection import cross_val_score

In [23]:
from sklearn.model_selection import train_test_split
seed = 42  # 乱数シード
X_cv, X_eval, y_cv, y_eval = train_test_split(X, y, test_size=0.25, random_state=42)
# 使用するチューニング対象外のパラメータ
params = {
    'objective': 'regression',  # 最小化させるべき損失関数
    'metric': 'rmse',  # 学習時に使用する評価指標(early_stoppingの評価指標にも同じ値が使用される)
    'random_state': seed,  # 乱数シード
    'boosting_type': 'gbdt',
    'n_estimators': 10000,  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
    'verbose': -1,  # これを指定しないと`No further splits with positive gain, best gain: -inf`というWarningが表示される
    'early_stopping_round': 10  # ここでearly_stoppingを指定
    }
# モデル作成
model = lgb.LGBMRegressor(**params)
# 学習時fitパラメータ指定
fit_params = {
    'eval_set': [(X_eval, y_eval)]  # early_stopping用のデータeval_setを渡す
    }
cv = KFold(n_splits=3, shuffle=True, random_state=seed)
scoring = 'neg_root_mean_squared_error'  # 評価指標をRMSEに指定

In [24]:
# # ベイズ最適化時の評価指標算出メソッド
# def bayes_objective(trial):
#     params = {
#         'reg_alpha': trial.suggest_float('reg_alpha', 0.0001, 0.1, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 0.0001, 0.1, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 6),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
#         'subsample': trial.suggest_float('subsample', 0.4, 1.0),
#         'subsample_freq': trial.suggest_int('subsample_freq', 0, 7),
#         'min_child_samples': trial.suggest_int('min_child_samples', 0, 10)
#     }
#     # モデルにパラメータ適用
#     model.set_params(**params)
#     # cross_val_scoreでクロスバリデーション
#     scores = cross_val_score(model, X, y, cv=cv,
#                              scoring=scoring, fit_params=fit_params, n_jobs=-1)
#     val = scores.mean()
#     return val

# # ベイズ最適化を実行
# study = optuna.create_study(direction='maximize',
#                             sampler=optuna.samplers.TPESampler(seed=42))
# study.optimize(bayes_objective, n_trials=400)

# # 最適パラメータの表示と保持
# best_params = study.best_trial.params
# best_score = study.best_trial.value
# print(f'最適パラメータ {best_params}\nスコア {best_score}')


In [25]:
best_params = {
    'reg_alpha': 0.030625843012809087, 
    'reg_lambda': 0.00017600143797012052, 
    'num_leaves': 3, 
    'colsample_bytree': 0.4238321137256774, 
    'subsample': 0.738201254772324, 
    'subsample_freq': 6, 
    'min_child_samples': 7
}
print(best_params)

{'reg_alpha': 0.030625843012809087, 'reg_lambda': 0.00017600143797012052, 'num_leaves': 3, 'colsample_bytree': 0.4238321137256774, 'subsample': 0.738201254772324, 'subsample_freq': 6, 'min_child_samples': 7}


In [26]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, mean_squared_error

In [27]:
# K分割交差検証
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
final_predictions = np.zeros(len(X))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, race_groups)):
    print(f"Fold {fold + 1}")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    race_val = race_groups.iloc[val_idx]

    # LightGBM用データセット作成
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
    val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=cat_features)

    # モデルのハイパーパラメータ
    # params = {
    #     'task':'train',
    #     'objective': 'regression', # 回帰
    #     'boosting':'gbdt', # 'gbdt'はGradient Boosting Decision TreeによるBoosting
    #     'metric': 'rmse', # C統計量と数学的に同じ値のため(https://note.com/yohei_okada/n/nd384bd62c6d5)
    #     'num_leaves': 64, # 決定木の最大葉ノード数（大きすぎると過学習）
    #     'max_depth': 7, # 決定木の深さ
    #     'min_data_in_leaf': 20, # それぞれの葉ノードに必要なデータ数（過学習の防止）
    #     'lambda_l2': 3, # L2正則化の強さを指定（大きいほど強く過学習が少なくなる）
    #     'random_state': 42, # 乱数シード．再現性の確保
    #     'verbose': -1, # 学習過程の表示レベル
    # }

    # 使用するチューニング対象外のパラメータ
    best_params = {
        'objective': 'regression',  # 最小化させるべき損失関数
        'metric': 'rmse',  # 学習時に使用する評価指標(early_stoppingの評価指標にも同じ値が使用される)
        'random_state': seed,  # 乱数シード
        'boosting_type': 'gbdt',
        'n_estimators': 10000,  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
        'verbose': -1,  # これを指定しないと`No further splits with positive gain, best gain: -inf`というWarningが表示される
        'early_stopping_round': 10,  # ここでearly_stoppingを指定
        'reg_alpha': 0.030625843012809087, 
        'reg_lambda': 0.00017600143797012052, 
        'num_leaves': 3, 
        'colsample_bytree': 0.4238321137256774, 
        'subsample': 0.738201254772324, 
        'subsample_freq': 6, 
        'min_child_samples': 7        
        }

    # モデルの訓練
    model = lgb.train(
        best_params, # Optunaで得たハイパーパラメータ
        train_data,
        valid_sets=[train_data, val_data],
        valid_names=['train', 'valid'],
        # early_stopping_rounds=50, # 検証スコアが改善されないラウンド数
        callbacks=[lgb.early_stopping(stopping_rounds=50)],  # コールバックで早期終了を設定（early_stopping_roundだとエラーが出たため）
        # verbose_eval=100, # 100回ごとに学習結果を表示
        num_boost_round=1000, # 学習ラウンド数
    )

    # 予測と評価
    y_val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    fold_score = stratified_c_index(y_val.values, y_val_pred, race_val.values) # Stratified C統計量の確認
    print(f"Stratified C-Index for Fold {fold + 1}: {fold_score}")

    final_predictions[val_idx] = y_val_pred

# 全体での評価スコア
overall_score = stratified_c_index(y.values, final_predictions, race_groups.values)

Fold 1
Training until validation scores don't improve for 50 rounds
Stratified C-Index for Fold 1: 0.6328639499316157
Fold 2
Training until validation scores don't improve for 50 rounds
Stratified C-Index for Fold 2: 0.6315890617103859
Fold 3
Training until validation scores don't improve for 50 rounds
Stratified C-Index for Fold 3: 0.6340325737180514
Fold 4
Training until validation scores don't improve for 50 rounds
Stratified C-Index for Fold 4: 0.6181804404118567
Fold 5
Training until validation scores don't improve for 50 rounds
Stratified C-Index for Fold 5: 0.6369427814361759


In [28]:
pred = model.predict(df_test)
pred

array([0.50524997, 0.58652755, 0.43969693])

**提出**

In [29]:
# # 提出用ファイルに代入
# sample_sub['prediction'] = pred
# sample_sub.to_csv('submission.csv', index=False)